<a href="https://colab.research.google.com/github/Didier06/IA_FABLAB/blob/main/Digit_recognition-opencv/cnn_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
!git clone https://github.com/Didier06/IA_FABLAB

Cloning into 'IA_FABLAB'...
remote: Enumerating objects: 97, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 97 (delta 6), reused 2 (delta 2), pack-reused 84
Receiving objects: 100% (97/97), 7.38 MiB | 22.29 MiB/s, done.
Resolving deltas: 100% (27/27), done.


**Vérification version de keras**

In [ ]:
import keras
keras.__version__

'2.15.0'

# Un premier réseau de convolution avec Keras

On utilise les couches `Conv2D` et `MaxPooling2D`.

Remarque imortante : le tenseur d'entrée du réseau de convolution avec keras doit avoir les dimensions suivantes : `(hauteur image, largeur  image, canal de l'image)`
Dans notre cas : `(28, 28, 1)`, format des images MNIST.  
Pour réaliser cette opération exécuter cette commande :  
`input_shape=(28, 28, 1)`.

In [ ]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

Affichons la structure du réseau :

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 13, 13, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 5, 5, 64)          0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 3, 3, 64)          36928     
                                                                 
Total params: 55744 (217.75 KB)
Trainable params: 5574

Les sorties des couches `Conv2D` et `MaxPooling2D` sont des tenseurs 3D avec les dimensions `(hauteur, largeur, canaux)`.
Le nombre de canaux est fixé par le 1er argument des couches  `Conv2D` (32 ou 64).

Dans l'étape suivante on aplati (Flatten) la couche 3D en une couche 1D puis on retrouve 2 couches "dense".
Avec pour la dernière une activaztion 'softmax'


In [ ]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

Nous avons réalisé une classification avec 10 sorties.  
Voici le modèle complet avec 93332 paramètres :

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 13, 13, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 5, 5, 64)          0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 3, 3, 64)          36928     
                                                                 
 flatten_1 (Flatten)         (None, 576)              

Le tenseur `(3, 3, 64)` est transformé en tenseur 1D de dimension `(576,)`, avant la classification avec les 2 couches 'Dense'  

Entrainons le réseau de convolution :

In [ ]:
from keras.datasets import mnist
from keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [15]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=5, batch_size=64)

Epoch 1/5
938/938 [==============================] - 69s 72ms/step - loss: 0.1770 - accuracy: 0.9442
Epoch 2/5
938/938 [==============================] - 52s 55ms/step - loss: 0.0453 - accuracy: 0.9859
Epoch 3/5
938/938 [==============================] - 51s 55ms/step - loss: 0.0317 - accuracy: 0.9902
Epoch 4/5
938/938 [==============================] - 50s 53ms/step - loss: 0.0249 - accuracy: 0.9923
Epoch 5/5
938/938 [==============================] - 48s 51ms/step - loss: 0.0195 - accuracy: 0.9940


Evaluons le modèle sur les données de test :

In [16]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
model.save('/content/IA_FABLAB/Digit_recognition-opencv/model/modelcnn.keras')

313/313 [==============================] - 3s 9ms/step - loss: 0.0284 - accuracy: 0.9920


FileNotFoundError: [Errno 2] No such file or directory: 'model/modelcnn.keras'

In [ ]:
test_acc

Le résultat est très satisfaisant .... est meilleur que celui obtenu avec le réseau dense.

**Prédictions avec une image dessinée sur paint.**

In [ ]:
# reading png image file
import matplotlib.pyplot as plt
import matplotlib.image as img
im = img.imread('/content/IA_FABLAB/Digit_recognition-opencv/images/digit_3.png')
plt.imshow(im, cmap=plt.cm.binary)
#print(im.shape)
#print(img.shape)

**Préparation de l'image dessinée et prédiction**

In [ ]:
from PIL import Image, ImageOps
import numpy as np
def preprocess_image(image_path):# préparation de l'image
    img = Image.open(image_path).convert('L')  # Convertir en niveau de gris grayscale
    img = ImageOps.invert(img)                 # inverse l'image pour qu'elle soit conforme aux images MNIST
    img = img.resize((28, 28))                 # Redimensionnement à 28x28
    img = np.array(img)                        # Convertir en tableau numpy
    img = img / 255.0                          # on normalise les valeurs des pixels
    img = img.reshape(1,28,28)                 # redimensionne en  (28,28) ( entrée du réseau cnn
    return img

processed_image = preprocess_image('/content/IA_FABLAB/Digit_recognition-opencv/images/digit_3.png')

predictions= model.predict(processed_image)# prévision

print(predictions.round(2)) # résultat
print("Chiffre prédit : ", predictions.argmax())# index  of the class with highest probability
